In [11]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
from nltk.stem import WordNetLemmatizer

In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [2]:
data=pd.read_csv("C:/Users/venky kishore/Downloads/Sentiment.csv")

In [3]:
data.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [4]:
(data.isnull().mean())*100

id                            0.000000
candidate                     0.692091
candidate_confidence          0.000000
relevant_yn                   0.000000
relevant_yn_confidence        0.000000
sentiment                     0.000000
sentiment_confidence          0.000000
subject_matter                2.350227
subject_matter_confidence     0.000000
candidate_gold               99.798140
name                          0.000000
relevant_yn_gold             99.769303
retweet_count                 0.000000
sentiment_gold               99.891861
subject_matter_gold          99.870233
text                          0.000000
tweet_coord                  99.848605
tweet_created                 0.000000
tweet_id                      0.000000
tweet_location               28.202725
user_timezone                31.742484
dtype: float64

In [5]:
data.drop(['sentiment_gold','subject_matter_gold','tweet_coord','relevant_yn_gold','candidate_gold','tweet_created','tweet_id'],axis=1,inplace=True)

In [6]:
dataf=data[["sentiment","text"]]

In [16]:
tweet_df =dataf[dataf['sentiment'] != 'neutral']

In [17]:
sentiment_label = tweet_df.sentiment.factorize()

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tweet = tweet_df.text.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(tweet)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [19]:
print(tokenizer.word_index)

{'w': 1, 'ww': 2, 'www': 3, 'wwww': 4}


In [20]:
print(tweet[0])
print(encoded_docs[0])

w 
[1]


In [21]:
print(padded_sequence[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers  import LSTM
from tensorflow.keras.layers  import Dense
from tensorflow.keras.layers  import Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [25]:
Finalmodel = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
347/347 [==============================] - 47s 129ms/step - loss: -5.2918 - accuracy: 0.1594 - val_loss: -11.7847 - val_accuracy: 0.1686
Epoch 2/5
347/347 [==============================] - 52s 149ms/step - loss: -11.8307 - accuracy: 0.1593 - val_loss: -19.7760 - val_accuracy: 0.1686
Epoch 3/5
347/347 [==============================] - 51s 147ms/step - loss: -17.8048 - accuracy: 0.1593 - val_loss: -27.7488 - val_accuracy: 0.1686
Epoch 4/5
347/347 [==============================] - 55s 160ms/step - loss: -23.7801 - accuracy: 0.1593 - val_loss: -35.5970 - val_accuracy: 0.1686
Epoch 5/5
347/347 [==============================] - 51s 146ms/step - loss: -29.7240 - accuracy: 0.1593 - val_loss: -43.4326 - val_accuracy: 0.1686


In [26]:
test_word ='He is a great leader.'
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
sentiment_label[1][prediction]

'Positive'